In [1]:
import pickle
from GAMERNet.rnet.networks.reaction_network import ReactionNetwork
import networkx as nx

# Loading the ReactionNetwork

In [2]:
with open('../scripts/C1_Cu100/rxn_net_bp.pkl', 'rb') as pickle_file:
    content = pickle.load(pickle_file)  # dict of elementary reactions

rxn_net = ReactionNetwork().from_dict(content)
print(rxn_net)

ReactionNetwork(43 adsorbed intermediates, 14 gas molecules, 105 elementary reactions)
Surface: Cu48(100)
Network Carbon cutoff: C1



## Searching paths from source to target

In [6]:
source = ['10111g']
target = ['14111g']
n_inter_steps = 8

all_paths_source_target = rxn_net.find_all_paths_from_sources_to_targets(source, target, n_inter_steps)
print('all_paths_source_target: ', all_paths_source_target)

all_paths_source_target:  {('10111g', '14111g'): [['10111g', '10111*', '10011*', '11011*', '12011*', '13011*', '14111*', '14111g'], ['10111g', '10111*', '10011*', '11011*', '12011*', '13111*', '14111*', '14111g'], ['10111g', '10111*', '10011*', '11011*', '12111*', '13111*', '14111*', '14111g'], ['10111g', '10111*', '10011*', '11111*', '12111*', '13111*', '14111*', '14111g'], ['10111g', '10111*', '11111*', '12111*', '13111*', '14111*', '14111g'], ['10111g', '10111*', '11112*', '11011*', '12011*', '13011*', '14111*', '14111g'], ['10111g', '10111*', '11112*', '11011*', '12011*', '13111*', '14111*', '14111g'], ['10111g', '10111*', '11112*', '11011*', '12111*', '13111*', '14111*', '14111g'], ['10111g', '10111*', '11112*', '12111*', '13111*', '14111*', '14111g'], ['10111g', '10111*', '11112*', '12112*', '12011*', '13011*', '14111*', '14111g'], ['10111g', '10111*', '11112*', '12112*', '12011*', '13111*', '14111*', '14111g'], ['10111g', '10111*', '11112*', '12112*', '13111*', '14111*', '14111g

## Searching paths from source to target through intermediate

In [7]:
source_inter = ['10111g', '02011g']
target_inter = '14111g'
intermediate = '13011*'
n_inter_steps_inter = 8

paths_thru_inter = rxn_net.find_paths_through_intermediate(source_inter[0], target_inter, intermediate, cutoff=n_inter_steps_inter)

proper_paths = []
count = 0
for paths in paths_thru_inter:
    # If path has repeated nodes,  removing the path from the list
    if len(paths) != len(set(paths)):
        count += 1
    else:
        proper_paths.append(paths)

paths_thru_inter:  [['10111g'], ['10111g', '10111*'], ['10111g', '10111*', '10011*'], ['10111g', '10111*', '10011*', '11011*'], ['10111g', '10111*', '10011*', '11011*', '12011*'], ['10111g', '10111*', '10011*', '11011*', '12011*', '13011*'], ['10111g', '10111*', '10011*', '11011*', '12011*', '13011*', '14011*'], ['10111g', '10111*', '10011*', '11011*', '12011*', '13011*', '14011*', '14011g'], ['10111g', '10111*', '10011*', '11011*', '12011*', '13011*', '14111*'], ['10111g', '10111*', '10011*', '11011*', '12011*', '13011*', '14111*', '13111*'], ['10111g', '10111*', '10011*', '11011*', '12011*', '13011*', '14111*', '13112*'], ['10111g', '10111*', '10011*', '11011*', '12011*', '13011*', '14111*', '14111g'], ['10111g', '10111*', '10011*', '11011*', '12011*', '13011*', '13112*'], ['10111g', '10111*', '10011*', '11011*', '12011*', '13011*', '13112*', '12112*'], ['10111g', '10111*', '10011*', '11011*', '12011*', '13011*', '13112*', '14111*'], ['10111g', '10111*', '10011*', '11011*', '12011*',

### Defining example paths

In [ ]:
example_path_1 = proper_paths[2]
example_path_2 = proper_paths[3]

In [ ]:
rxn_graph = rxn_net.gen_graph(show_steps=True)

for edge in list(rxn_graph.edges):
    rxn_graph.add_edge(edge[1], edge[0])

### Storing the necessary information for example_path_1

In [ ]:
full_path_1 = []
for inter_1, inter_2 in zip(example_path_1[:-1], example_path_1[1:]):
    # Matching the nodes in the path to the nodes in the graph
    node_1 = rxn_graph.nodes[inter_1]
    node_2 = rxn_graph.nodes[inter_2]
    # Finding Elementary reactions that connect the nodes 
    elem_react = nx.shortest_path(rxn_graph, inter_1, inter_2)
    # Appending the first node only for the first iteration
    if len(full_path_1) == 0:
        full_path_1.append(node_1)
    full_path_1.append(elem_react)
    full_path_1.append(node_2)

### Storing the necessary information for example_path_2

In [ ]:
full_path_2 = []
for inter_1, inter_2 in zip(example_path_2[:-1], example_path_2[1:]):
    # Matching the nodes in the path to the nodes in the graph
    node_1 = rxn_graph.nodes[inter_1]
    node_2 = rxn_graph.nodes[inter_2]
    # Finding Elementary reactions that connect the nodes 
    elem_react = nx.shortest_path(rxn_graph, inter_1, inter_2)
    # Appending the first node only for the first iteration
    if len(full_path_2) == 0:
        full_path_2.append(node_1)
    full_path_2.append(elem_react)
    full_path_2.append(node_2)

### Generating list with only the elementary reaction steps for example_path_1

In [ ]:
# Generating a new Reaction Network from full_path

elem_react_list = []
for node in full_path_1:
    if isinstance(node, dict):
        continue
    else:
        elem_react_list.append(node)

elem_react_for_rxn_net = []
for elem_react in elem_react_list:
    elem_react_rxn_net = rxn_net.search_reaction_by_code(elem_react[1])
    elem_react_for_rxn_net.append(elem_react_rxn_net)



### Generating list with only the elementary reaction steps for example_path_2

In [ ]:
elem_react_list_2 = []
for node in full_path_2:
    if isinstance(node, dict):
        continue
    else:
        elem_react_list_2.append(node)

elem_react_for_rxn_net_2 = []
for elem_react in elem_react_list_2:
    elem_react_rxn_net = rxn_net.search_reaction_by_code(elem_react[1])
    elem_react_for_rxn_net_2.append(elem_react_rxn_net)


### Obtaining the Intermediates participating in example_path_1

In [ ]:
all_inter_in_react_1 = []
for elem_react in elem_react_for_rxn_net:
    all_inter_in_react_1.extend([x for x in elem_react.reactants])
    all_inter_in_react_1.extend([x for x in elem_react.products])


In [ ]:
# # For Carbon-less intermediates, search the associated elementary reactions and closed shell intermediates
# carbonless_inter = []
# for inter in all_inter_in_react_1:
#     if '1' not in inter.code[0] and '00000' not in inter.code:
#         carbonless_inter.append(inter)
# carbonless_inter = list(set(carbonless_inter))


In [ ]:
# for inter in carbonless_inter:
#     rxn_list = rxn_net.search_reaction_by_inter([inter])
#     for rxn in rxn_list:
#         if rxn.r_type == 'adsorption':
#             print(rxn.code)
#             print(rxn.reactants)
#             print(rxn.products)

### Obtaining the Intermediates participating in example_path_2

In [ ]:
all_inter_in_react_2 = []
for elem_react in elem_react_for_rxn_net_2:
    all_inter_in_react_2.extend([x for x in elem_react.reactants])
    all_inter_in_react_2.extend([x for x in elem_react.products])

### Generating a new ReactionNetwork for example_path_1

In [ ]:
# Generating the new rxn net

inter_for_rxn_net = {}
for inter in all_inter_in_react_1:
    inter_rxn_net_obj = rxn_net.intermediates[inter]
    inter_for_rxn_net[inter] = inter_rxn_net_obj

new_rxn_net_1 = ReactionNetwork(intermediates=inter_for_rxn_net, reactions=elem_react_for_rxn_net)
new_rxn_net_1

### Generating a new ReactionNetwork for example_path_1

In [ ]:
# Generating the new rxn net

inter_for_rxn_net_2 = {}
for inter in all_inter_in_react_2:
    inter_rxn_net_obj = rxn_net.intermediates[inter]
    inter_for_rxn_net_2[inter] = inter_rxn_net_obj

new_rxn_net_2 = ReactionNetwork(intermediates=inter_for_rxn_net_2, reactions=elem_react_for_rxn_net_2)
new_rxn_net_2

### Merging both paths (ReactionNetwork merge)

In [ ]:
# Merging the two rxn nets

new_rxn_net = new_rxn_net_1 + new_rxn_net_2

### Plotting the dotgraph of the new Network

In [ ]:
# Generating the plot
new_rxn_net.write_dotgraph('.', 'new_rxn_net_merge.png', del_surf=False, show_steps=True)